In [ ]:
import os
import concurrent
import pandas_datareader.data as web
import pandas as pd
import datetime
from datetime import timedelta
import concurrent.futures
from concurrent.futures import wait, ALL_COMPLETED
import yfinance as yf
import urllib
import urllib.request
import time
from finquant.portfolio import build_portfolio

import pandas_datareader as pdr

from ipywidgets import interactive
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import pycorrelate as pyc
import seaborn as sns
from yahoofinancials import YahooFinancials
#from sympy import symbols, solve

import pandas_market_calendars as mcal
import re


from scipy import stats # For in-built method to get PCC
from scipy.ndimage.interpolation import shift

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
#import statsmodels.formula.api as sm
import scipy.stats  as stats
from itertools import cycle

from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from sklearn.datasets import load_iris
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from dask.distributed import Client
from dask.distributed import as_completed

In [ ]:
def split_sequences(sequences, n_steps_in, n_steps_out):

    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)

    return np.array(X), np.array(y)

def crosscorrelation(x, y, maxlag, mode='corr'):
	"""
	Cross correlation with a maximum number of lags.

	`x` and `y` must be one-dimensional numpy arrays with the same length.

	This computes the same result as
		numpy.correlate(x, y, mode='full')[len(a)-maxlag-1:len(a)+maxlag]

	The return vaue has length 2*maxlag + 1.
	"""
	py = np.pad(y.conj(), 2*maxlag, mode='constant')
	T = np.lib.stride_tricks.as_strided(py[2*maxlag:], shape=(2*maxlag+1, len(y) + 2*maxlag),
				   strides=(-py.strides[0], py.strides[0]))
	px = np.pad(x, maxlag, mode='constant')
	if mode == 'dot':       # get lagged dot product
		return T.dot(px)
	elif mode == 'corr':    # gets Pearson correlation
		return (T.dot(px)/px.size - (T.mean(axis=1)*px.mean())) / \
			   (np.std(T, axis=1) * np.std(px))

def unique(list1):

    # intilize a null list
    unique_list = []

    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)

    return(unique_list)

def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value
        
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [ ]:
#frequency = output_variable.value
frequency = "Q"

wd = os.getcwd()

w=52*15
#start = datetime.datetime(2010, 1, 1)

#end = datetime.datetime(2013, 1, 27)
end_date = datetime.date.today()

start_date = end_date - timedelta(weeks=w)

pd.set_option('display.max_columns', None) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows', None) #replace n with the number of rows you want to see completely

#cores = int(len(os.sched_getaffinity(0)))

print(end_date)
print(start_date)

one_week_end = end_date - 5 * pd.tseries.offsets.BDay()
#one_week_end = one_week_end.strftime("%Y-%m-%d")

# Create a calendar
nyse = mcal.get_calendar('NYSE')

# Show available calendars
print(mcal.get_calendar_names())
early = nyse.schedule(start_date, end_date)

nstocks = 300

In [ ]:
url = "ftp://ftp.nasdaqtrader.com/symboldirectory/nasdaqtraded.txt"

urllib.request.urlretrieve(url, "nasdaqtraded.txt")
urllib.request.urlretrieve(url, "mfundslist.txt")
urllib.request.urlretrieve(url, "bonds.txt")

df1 = pd.read_csv("nasdaqtraded.txt", sep="|")[0:-1]
df2 = pd.read_csv("mfundslist.txt", sep="|")[0:-1]
df3 = pd.read_csv("bonds.txt", sep="|")[0:-1]

# combined = pd.concat([df1['Symbol'],df2['Symbol'],df3['Symbol']],axis=0)

# process symbols for bad characters
BAD_CHARS = ["$", "."]
# pat = '|'.join(['({})'.format(re.escape(c)) for c in BAD_CHARS])
# cleaned = unique(combined.replace(BAD_CHARS,'-'))

# choose size
size = nstocks
# stocks = list(df1["Symbol"].sample(n=int(size/3)))
stocks = list(
    df1["Symbol"]
    .replace(".", "-")
    .replace("\\$", "-P", regex=True)
    .sample(n=int(size / 3))
)
mfunds = list(
    df2["Symbol"]
    .replace(".", "-")
    .replace("\\$", "-P", regex=True)
    .sample(n=int(size / 3))
)
bonds = list(
    df3["Symbol"]
    .replace(".", "-")
    .replace("\\$", "-P", regex=True)
    .sample(n=int(size / 3))
)
symbols = list(set(stocks + mfunds + bonds))  # unique(stocks + mfunds + bonds)
# symbols = unique(stocks)

In [ ]:
def dl_data (npa_):
    name = npa_[0]
    s_ = npa_[1]
    e_ = npa_[2]    
    
    subset = pd.DataFrame()

    subset = yf.download(name, start=s_.strftime('%Y-%m-%d'), end=e_.strftime('%Y-%m-%d'), auto_adjust=True,progress=False,show_errors=False, threads = False).iloc[:, :6].dropna(axis=0, how='any')
    subset = subset[s_.strftime('%Y-%m-%d'):e_.strftime('%Y-%m-%d')]

    if len(subset) > 2:
        return (subset)
    else:
        return(0)
    

In [ ]:
client = Client('192.168.3.100:8786',timeout=3)

client.ncores

npa = []

for s in range(0,len(symbols)):
    npa.append([symbols[s],start_date,one_week_end])

#results = await client.gather(futures, asynchronous=True)
    
future = client.map(dl_data,npa)

results = []
#best = -1
for f in as_completed(future):
    if(f.status=="error"):
        results.append("error")
    else:
        results.append(f.result())
    #print(f.result())

client.close()

vetted_symbols = []

for r in range(0,len(results)):
    if(isinstance(results[r], pd.DataFrame)):
        vetted_symbols.append(symbols[r])

In [ ]:
len(vetted_symbols)

In [ ]:
npa = []

for s in range(0,len(vetted_symbols)):
    npa.append([vetted_symbols[s],start_date,end_date])
    
client = Client('192.168.3.100:8786',timeout=3)

future = client.map(dl_data,npa)

symbols_completed = []
#best = -1
for f in as_completed(future):
    if(f.status=="error"):
        symbols_completed.append("error")
    else:
        symbols_completed.append(f.result())
    #print(f.result())

client.close()



In [ ]:
symbols_completed_ = pd.DataFrame()

#commodities_includes = []
#names = []

#completed_2 = []

for x in range(0,len(symbols_completed)):
    
    if isinstance(symbols_completed[x], pd.DataFrame):
        #print(commodities[x])
        #print("Not")
        #commodities_includes.append(True)
        values = symbols_completed[x]
        values.index = pd.to_datetime(values.index)
        values = values.resample(frequency).mean().dropna()
        values['Symbol'] = symbols[x]
        values = values.loc[~values.index.duplicated(keep='last')]
        values = values.reset_index()
        #names.append(completed[x])

        #if(len(values))
        symbols_completed_ = pd.concat([symbols_completed_,values], axis=0)        
        
    #else:
        #print("None")
        

In [ ]:
len(symbols_completed_)

In [ ]:
symbols_pvt = pd.pivot_table(symbols_completed_, columns=['Symbol'], index = 'Date', values='Close')

In [ ]:
# Show available calendars
# print(mcal.get_calendar_names())
'''
vetted_symbols = list(
    pf_pre.data.loc[
        (np.intersect1d(list(pf_pre.data.index.strftime("%Y-%m-%d")), early.index.strftime('%Y-%m-%d')))
    ]
    .head(-1)
    .tail(-1)
    .dropna(axis=1)
    .columns
)
'''

In [ ]:

#Ultra-Low-Sulfur No. 2 Diesel Fuel Prices: Los Angeles (WDFUELLA)
#US Regular All Formulations Gas Price (GASREGW)
#Trade Weighted U.S. Dollar Index: Broad, Goods and Services (DTWEXBGS)


etf_commodities = ['DBO','CORN', 'WEAT', 'SOYB', 'JO', 'SGG', 'BAL', 'COW', 'MOO', 'TAGS', 'KOL' ]
#Gold, Silver, Platinum, Copper, Paladium, Aluminum, Iron, Steel
etf_metals = ['IAU', 'SLV', 'PGM', 'JJC', 'PALL', 'JJU', 'IFUNX', 'SLX']
#US dollar, European Euro, Japanese yen, Pound sterling, Australian dollar, Canadian dollar, Swiss franc, Chinese Yuan Renminbi, Swedish Krona, Peso, India
#defunct: Russia: XRU, Mexico: FXM
etf_foreign_exchanges = ['UUP','FXE','FXY','FXB','FXA','FXC','FXF','CYB', 'FXS', 'INR']
#residential, Ishares all NAmerica
etf_real_estate = ['REZ', 'IYR']
#Russia, Germany, UK, Japan, China, Euro, Euro, Brazil, Latin America, Mexico, India
etf_economies = ['ERUS','EWG','EWU','EWJ','MCHI','EZU','IEUR','EWZ','ILF','EWW','INDA']
#Ishares Investment Grade, IShares core aggregate Investment grade, Short, Total, 1-5 Years, 5-10 Years, 10 Years, Gov/Credit
#defunct:

etf_spdr_indexes = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
etf_dow_components = ['MMM','AXP','AMGN','AAPL','BA','CAT','CVX','CSCO','KO','DOW','GS','HD','HON','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PG','CRM','TRV','UNH','VZ','V','WMT','WBA','DIS']

etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

crypto = ['BTC-USD','ETH','RPL','BCH','EOS','LTC']

FRED_Indicators = [
"ASPUS",\
"AWHAETP",\
"B432RG3A086NBEA",\
"BAA10Y",\
"BACDINA066MNFRBNY",\
"BACTSAMFRBDAL",\
"BAMLC0A0CM",\
"BAMLCC0A1AAATRIV",\
"BAMLCC0A4BBBTRIV",\
"BAMLH0A3HYC",\
"BAMLHYH0A0HYM2TRIV",\
"BOGMBASE",\
"BOGZ1FA105015103Q",\
"BOGZ1FA145050005Q",\
"BOGZ1FA205050005Q",\
"BOGZ1FA315050005Q",\
"BOGZ1FA385050005Q",\
"BOGZ1FL105015105Q",\
"BUSAPPWNSACA",\
"CASTHPI",\
"CES0500000003",\
"CES4348400001",\
"CFNAI",\
"CFNAIDIFF",\
"CFSBCACTIVITYMFG",\
"CILACBQ158SBOG",\
"CIVPART",\
"COMREPUSQ159N",\
"CP",\
"CPALTT01USQ657N",\
"CPIAUCSL",\
"CSCICP03USM665S",\
"CSUSHPINSA",\
"CUSR0000SEHA",\
"CUUR0000SEHA",\
"DALLCIACBEP",\
"DCOILBRENTEU",\
"DCOILWTICO",\
"DEXCHUS",\
"DEXUSUK",\
"DFF",\
"DFII10",\
"DGORDER",\
"DGS1",\
"DGS10",\
"DGS1MO",\
"DGS2",\
"DGS30",\
"DGS3MO",\
"DGS5",\
"DRBLACBS",\
"DRCCLACBS",\
"DRSFRMACBS",\
"DTB3",\
"DTWEXBGS",\
"EMRATIO",\
"ERENTUSQ176N",\
"ETOTALUSQ176N",\
"FEDFUNDS",\
"FGCCSAQ027S",\
"FPCPITOTLZGUSA",\
"GASREGW",\
"GDPC1",\
"GFDEBTN",\
"GFDEGDQ188S",\
"GOLDAMGBD228NLBM",\
"GVZCLS",\
"HDTGPDUSQ163N",\
"HOSMEDUSM052N",\
"IC4WSA",\
"ICSA",\
"IIPUSNETIQ",\
"INDPRO",\
"INTDSRUSM193N",\
"IRLTLT01USM156N",\
"KCFSI",\
"LES1252881600Q",\
"LEU0252918500Q",\
"LFWA64TTUSM647S",\
"LRUN64TTUSQ156S",\
"LXXRCSA",\
"M1",\
"M1V",\
"M2V",\
"MABMM301USM189S",\
"MANMM101USA189S",\
"MEFAINUSA672N",\
"MEHOINUSA672N",\
"MEPAINUSA672N",\
"MICH",\
"MORTGAGE30US",\
"MPRIME",\
"MSPUS",\
"MZMSL",\
"NFCI",\
"NROU",\
"NYSTHPI",\
"PAYEMS",\
"PERMIT",\
"POP",\
"POPTOTUSA647NWDB",\
"PPIACO",\
"PRFI",\
"PSAVERT",\
"Q10096USQ144NNBR",\
"RECPROUSM156N",\
"RSAHORUSQ156S",\
"SPCS20RSA",\
"STLFSI2",\
"T10Y2Y",\
"T10Y3M",\
"T10YIE",\
"T5YIE",\
"T5YIFR",\
"TB3MS",\
"TCU",\
"TDSP",\
"TEDRATE",\
"TOTALSA",\
"TREAST",\
"TTLHH",\
"TWEXB",\
"UMCSENT",\
"U6RATE",\
"UNRATE",\
"USPHCI",\
"USREC",\
"USROA",\
"USROE",\
"USSLIND",\
"USSTHPI",\
"VIXCLS",\
"VXVCLS",\
"WDFUELLA",\
"WGS3MO",\
"WILLLRGCAPVAL",\
"WILLMICROCAPPR",\
"WPU0911",\
"WTB3MS"    
]
#FRED_Indicators = ["CPALTT01USQ657N","PAYEMS","IRLTLT01USM156N","MABMM301USM189S","LFWA64TTUSM647S","MANMM101USA189S","MICH","UMCSENT","CSCICP03USM665S","DGS10","DTB3","DGS3MO","CASTHPI","GDPC1","CIVPART","POPTOTUSA647NWDB","MEHOINUSA672N","HOSMEDUSM052N","MORTGAGE30US","TTLHH","CSUSHPINSA","EMRATIO","CPIAUCSL","PSAVERT","LRUN64TTUSQ156S","USSTHPI","NYSTHPI","M2V","GFDEBTN","DFII10","GFDEGDQ188S","CUSR0000SEHA","ETOTALUSQ176N","ERENTUSQ176N","RECPROUSM156N","T5YIFR","BAMLHYH0A0HYM2TRIV","BAMLCC0A1AAATRIV","GVZCLS","DGS1","BAMLCC0A4BBBTRIV","VXVCLS","IC4WSA","WILLMICROCAPPR","WILLLRGCAPVAL","CFNAIDIFF","MZMSL","KCFSI","T5YIE","TOTALSA","USSLIND","AWHAETP","CES0500000003","TCU","WTB3MS","WGS3MO","TWEXB","DEXCHUS","DEXUSUK","CILACBQ158SBOG","CES4348400001","FEDFUNDS","TDSP","PERMIT","GFDEGDQ188S","CP","PRFI","DRSFRMACBS","DRCCLACBS","DRBLACBS","DALLCIACBEP","USROA","USROE","RSAHORUSQ156S","MEFAINUSA672N","COMREPUSQ159N","HDTGPDUSQ163N","POP","NROU","FGCCSAQ027S","TEDRATE", "VIXCLS", "NFCI","INDPRO","LES1252881600Q","CUUR0000SEHA","LEU0252918500Q","BAA10Y","BAMLC0A0CM","BAMLH0A3HYC","BOGMBASE","DCOILBRENTEU","DCOILWTICO","DFF","DGS1MO","DGS30","DGS5","FPCPITOTLZGUSA","GOLDAMGBD228NLBM","ICSA","INTDSRUSM193N","M1","M1V","MPRIME","PPIACO","SPCS20RSA","STLFSI2","T10Y2Y","T10Y3M","TB3MS","TREAST","UNRATE","WPU0911"]

Indexes = ['^SP500TR', '^GSPC', 'QQQ', 'DIA', 'VTWO', 'RUA','W5000FLT']

ManualStocks = ['VOO','SPY']
ManualStocks.extend(vetted_symbols)

etf_indexes_and_Crypto_list = [Indexes, ManualStocks, etf_commodities, etf_metals, etf_foreign_exchanges, etf_real_estate, etf_economies, etf_bonds, etf_muni_bonds, etf_treasuries, crypto, etf_spdr_indexes, etf_dow_components]

commodities = []
for sublist in etf_indexes_and_Crypto_list:
    for val in sublist:
        commodities.append(val)

#pool2 = concurrent.futures.ProcessPoolExecutor(cores)

completed = []
def dl(name):
    subset = yf.download(name, start=start_date, end=end_date, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    subset = subset[start_date.strftime('%Y-%m-%d'):end_date.strftime('%Y-%m-%d')]
    #sleep(1)
    if len(subset) != 0:
        completed.append(name)
        return (subset)
    
def dl2(assets):
    #https://webcache.googleusercontent.com/search?q=cache:Em9Ge5B9ue8J:https://stackoverflow.com/questions/62614282/is-the-yfinance-module-broken-i-keep-getting-a-jsondecodeerror+&cd=3&hl=en&ct=clnk&gl=us

    yahoo_financials = YahooFinancials(assets)

    #sleep(1)
    data = yahoo_financials.get_historical_price_data(start_date=start_date.strftime('%Y-%m-%d'), end_date=end_date.strftime('%Y-%m-%d'), time_interval='daily')
    return(data)

In [ ]:
npa = []

for s in range(0,len(commodities)):
    npa.append([commodities[s],start_date,one_week_end])

client = Client('192.168.3.100:8786',timeout=3)

future = client.map(dl_data,npa)

completed_commodities = []
#best = -1
for f in as_completed(future):
    if(f.status=="error"):
        completed_commodities.append("error")
    else:
        completed_commodities.append(f.result())
    #print(f.result())

client.close()

vetted_commodities = []

for r in range(0,len(completed_commodities)):
    if(isinstance(completed_commodities[r], pd.DataFrame)):
        vetted_commodities.append(commodities[r])

#change end date
npa = []

for s in range(0,len(commodities)):
    npa.append([commodities[s],start_date,end_date])

client = Client('192.168.3.100:8786',timeout=3)

future = client.map(dl_data,npa)

commodities_data =[]
#best = -1
for f in as_completed(future):
    if(f.status=="error"):
        commodities_data.append("error")
    else:
        commodities_data.append(f.result())
    #print(f.result())    

client.close()
            

In [ ]:
len(vetted_commodities)

In [ ]:
commodities_ = pd.DataFrame()

#commodities_includes = []
#names = []

#completed_2 = []

for x in range(0,len(completed_commodities)):
    
    if isinstance(completed_commodities[x], pd.DataFrame):
        #print(commodities[x])
        #print("Not")
        #commodities_includes.append(True)
        values = completed_commodities[x]
        values.index = pd.to_datetime(values.index)
        values = values.resample(frequency).mean().dropna()
        values['Symbol'] = commodities[x]
        values = values.loc[~values.index.duplicated(keep='last')]
        values = values.reset_index()
        #names.append(completed[x])

        commodities_ = pd.concat([commodities_,values], axis=0)        
        
    #else:
        #print("None")
        

In [ ]:
commodities_pvt = pd.pivot_table(commodities_, columns=['Symbol'], index = 'Date', values='Close')

In [ ]:
#commodities_pvt.describe()

In [ ]:
def Fred_Data(npa_):
    name = npa_[0]
    s_ = npa_[1]
    e_ = npa_[2]    
    
    '''
    subset = pd.DataFrame()

    subset = yf.download(name, start=s_.strftime('%Y-%m-%d'), end=e_.strftime('%Y-%m-%d'), auto_adjust=True,progress=False,show_errors=False, threads = False).iloc[:, :6].dropna(axis=0, how='any')
    subset = subset[s_.strftime('%Y-%m-%d'):e_.strftime('%Y-%m-%d')]

    if len(subset) != 0:
        return (subset)
    else:
        return(0)
    '''
    temp = web.get_data_fred(str(name), s_.strftime('%Y-%m-%d'), e_.strftime('%Y-%m-%d'))
    temp.index = pd.to_datetime(temp.index)
    temp = temp.resample(frequency).mean().dropna()
    return(temp)

npa = []

for s in range(0,len(FRED_Indicators)):
    npa.append([FRED_Indicators[s],start_date,one_week_end])

client = Client('192.168.3.100:8786',timeout=3)

future = client.map(Fred_Data,npa)

FRED_completed = []

for f in as_completed(future):
    if(f.status=="error"):
        FRED_completed.append("error")
    else:
        FRED_completed.append(f.result())
    #print(f.result())

client.close()



npa = []

for s in range(0,len(FRED_completed)):
    npa.append([FRED_completed[s],start_date,one_week_end])

client = Client('192.168.3.100:8786',timeout=3)

future = client.map(Fred_Data,npa)

FRED_set = []

for f in as_completed(future):
    if(f.status=="error"):
        FRED_completed.append("error")
    else:
        FRED_completed.append(f.result())
    #print(f.result())

client.close()


In [ ]:
FRED_set_ = pd.DataFrame()
#FRED_set_ = []

#commodities_includes = []
#names = []

#completed_2 = []

for x in range(0,len(FRED_completed)):
    
    if isinstance(FRED_completed[x], pd.DataFrame):
        #print(commodities[x])
        #print("Not")
        #commodities_includes.append(True)
        
        values = FRED_completed[x]
        values.index = pd.to_datetime(values.index)
        values = values.resample(frequency).mean().dropna()
        
        values = values.loc[~values.index.duplicated(keep='last')]
        values = values.reset_index()
        values['Symbol'] = values.columns[1]
        values.columns = ['DATE','Value','Symbol']
        #names.append(completed[x])

        #FRED_set_.append(values)#
        FRED_set_ = pd.concat([FRED_set_,values], axis=0)        
        
    #else:
        #print("None")
        

In [ ]:
FRED_set_.columns

In [ ]:
FRED_set_pvt = pd.pivot_table(FRED_set_, columns=['Symbol'], index = 'DATE', values='Value')

In [ ]:
#FRED_set_pvt

In [ ]:
combined_set = pd.concat([FRED_set_pvt,symbols_pvt,commodities_pvt],axis=1)

drops = combined_set.describe().loc['count'].index[combined_set.describe().loc['count']<(len(combined_set)*.90)]
print(drops)
filtered = combined_set.columns.tolist()

for d in drops:
    #print(d)
    filtered.remove(d)
#filtered.remove(drops.tolist())
#combined_set[filtered].to_csv("/mnt/distvol/combined_set.csv")

#drop first/last row
all_data = combined_set[filtered]
all_data = all_data.iloc[:-1,:]
all_data = all_data.iloc[1:,:]

#all_data.loc[:, (all_data != all_data.iloc[0]).any()] 
#*** should fix duplications
all_data = all_data.loc[:,~all_data.columns.duplicated()]

all_data.to_csv(re.sub('code', 'data', wd)+"\combined_set.csv",index=True, index_label='Date')
#filtered
all_data.describe()

In [ ]:
#display(all_data)
save_list = all_data.replace('', np.nan).count()>(len(FRED_set_pvt)*.9)

In [ ]:
save_list_ = (save_list.index[save_list.values])

In [ ]:
all_data_filtered = all_data[save_list_.values]

all_data_filtered.replace('', np.nan).count()
truncatedData = all_data_filtered.reindex(early.asfreq(frequency, method='pad').index).interpolate(method='time')
#len(truncatedData.dropna(axis=0).columns)
#truncatedData = truncatedData.replace([np.inf, -np.inf, np.NaN], 0).interpolate(method='time')

In [ ]:
all_data_filtered

In [ ]:
all_data_filtered.to_csv("all_data.csv",index_label="Date")